In [29]:
import pandas as pd
import quandl, math, datetime
import numpy as np
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import plotly.express as px
import pickle

df = quandl.get("WIKI/GOOGL")

df.tail()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2018-03-21,1092.57,1108.70,1087.21,1094.00,1990515.0,0.0,1.0,1092.57,1108.70,1087.21,1094.00,1990515.0
2018-03-22,1080.01,1083.92,1049.64,1053.15,3418154.0,0.0,1.0,1080.01,1083.92,1049.64,1053.15,3418154.0
2018-03-23,1051.37,1066.78,1024.87,1026.55,2413517.0,0.0,1.0,1051.37,1066.78,1024.87,1026.55,2413517.0
2018-03-26,1050.60,1059.27,1010.58,1054.09,3272409.0,0.0,1.0,1050.60,1059.27,1010.58,1054.09,3272409.0
2018-03-27,1063.90,1064.54,997.62,1006.94,2940957.0,0.0,1.0,1063.90,1064.54,997.62,1006.94,2940957.0


In [30]:
df = df[['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume',]]

In [31]:
df['HL_PCT'] = (df['Adj. High'] - df['Adj. Close']) / df['Adj. Close'] * 100.0

In [32]:
df['PCT_change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100.0	

In [33]:
df = df[['Adj. Close', 'HL_PCT', 'PCT_change', 'Adj. Volume']]
df.tail()

,Adj. Close,HL_PCT,PCT_change,Adj. Volume
Date,,,,
2018-03-21,1094.00,1.343693,0.130884,1990515.0
2018-03-22,1053.15,2.921711,-2.487014,3418154.0
2018-03-23,1026.55,3.918952,-2.360729,2413517.0
2018-03-26,1054.09,0.491419,0.332191,3272409.0
2018-03-27,1006.94,5.720301,-5.353887,2940957.0


In [34]:
forecast_col = 'Adj. Close'

In [35]:
df.fillna(-99999, inplace=True)
dff = df

In [36]:
forecast_out = int(math.ceil(0.01*len(df))) # 1% of the data (aka. 1 day out)

In [37]:
df['label'] = df[forecast_col].shift(-forecast_out) # grabs only the data that is 1% of the data

In [38]:
df.dropna(inplace=True)
df.head()

,Adj. Close,HL_PCT,PCT_change,Adj. Volume,label
Date,,,,,
2004-08-19,50.322842,3.712563,0.324968,44659000.0,69.078238
2004-08-20,54.322689,0.710922,7.227007,22834300.0,67.839414
2004-08-23,54.869377,3.729433,-1.227880,18256100.0,68.912727
2004-08-24,52.597363,6.417469,-5.726357,15247300.0,70.668146
2004-08-25,53.164113,1.886792,1.183658,9188600.0,71.219849


In [39]:
X = np.array(df.drop(['label'], 1)) # features (everything except label) \
y = np.array(df['label']) # labels (only label) \

X = preprocessing.scale(X) \

df.dropna(inplace=True)

y = np.array(df['label'])

C:\Users\g4bri\AppData\Local\Temp\ipykernel_16992\1774715894.py:1: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



In [40]:
len(X), len(y)

(3389, 3389)

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [42]:
clf = LinearRegression(n_jobs=-1)
clf.fit(X_train, y_train)

LinearRegression(n_jobs=-1)

In [43]:
acc = clf.score(X_test, y_test)
acc

0.9767350629596894

In [44]:
clf = svm.SVR() #kernel= 'poly'
clf.fit(X_train, y_train)

SVR()

In [45]:
acc = clf.score(X_test, y_test)
acc

0.8029845430731225

In [46]:
forecast_out = int(math.ceil(0.1*len(dff)))
dff['label'] = dff[forecast_col].shift(-forecast_out)

X = np.array(dff.drop(['label'], 1)) # features (everything except label) 
X = preprocessing.scale(X)
X = X[:-forecast_out] # everything except the last 30% of the data
X_lately = X[-forecast_out:] # the last 30% of the data

dff.dropna(inplace=True)

y = np.array(dff['label'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

C:\Users\g4bri\AppData\Local\Temp\ipykernel_16992\2823232571.py:4: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



In [47]:
# clf = LinearRegression(n_jobs=-1)
# clf.fit(X_train, y_train)
# with open ('linearregression.pickle', 'wb') as f:
#     pickle.dump(clf, f)

clf = pickle.load(open('linearregression.pickle', 'rb'))
acc = clf.score(X_test, y_test)
acc

0.8690485045469309

In [48]:
forecast_set = clf.predict(X_lately)

forecast_set, acc, forecast_out

(array([ 688.72657145,  683.46267277,  681.93322439,  698.85906239,
         691.78543657,  689.74394844,  684.83617564,  679.26977703,
         674.58127893,  692.81073785,  685.63041191,  680.09428424,
         673.95418289,  677.48298282,  680.47491615,  695.63409584,
         689.45230719,  702.55362614,  701.89698271,  694.90786118,
         697.48164933,  690.63495582,  682.63325709,  674.67201208,
         676.33418616,  680.32854539,  683.20972952,  683.89411737,
         679.23249182,  678.2729316 ,  694.61012014,  717.57664618,
         736.51546013,  729.22321102,  752.9542259 ,  869.04903076,
         860.14519975,  876.27379732,  881.1163462 ,  850.5642259 ,
         826.13312379,  828.41645351,  818.68674668,  817.70443564,
         829.04795156,  817.36591927,  827.26299514,  823.84591875,
         844.62457079,  830.18048652,  823.82205983,  827.77786954,
         861.16141046,  855.54943959,  855.1565874 ,  857.38664207,
         860.44773644,  857.73132675,  865.78934

In [49]:
dff['Forecast'] = np.nan

last_date = dff.iloc[-1].name
last_unix = last_date.timestamp()
day = 86400
next_unix = last_unix + day

for i in forecast_set:
    next_date = datetime.datetime.fromtimestamp(next_unix)
    next_unix += day
    dff.loc[next_date] = [np.nan for _ in range(len(dff.columns)-1)]+[i]

In [50]:
fig = px.line(dff, x=dff.index, y=['Adj. Close', 'Forecast'], title='Google Stock Price Prediction', labels={'value':'Price', 'variable':'Stock Price', 'index':'Date'}, template='plotly_dark')
fig.show()